# Question Answering

In [ ]:
%pip install langchain openai chromadb tiktoken pypdf llama-index

In [18]:
import tomli, os
with open("../.streamlit/secrets.toml","rb") as f:
    secrets = tomli.load(f)
os.environ["OPENAI_API_KEY"] = secrets["OPENAI_API_KEY"]

In [11]:
# Don't forget to load your OPENAI_API_KEY as env variable
from openai import OpenAI
openai = OpenAI()
def ask(question):
    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": question}
        ]
        )
    return completion.choices[0].message.content

In [3]:
ask("What is the capital of France?")

'The capital of France is Paris.'

## Traditional search

If you ask chatGPT directly for an evolving topic, like twitter, you will depend on the training set of the model. If the model was trained on a dataset that is not up to date, the model will not be able to answer your question. 

In [4]:
prompt = 'who is the CEO of twitter?'
ask(prompt)

'As of September 2021, the CEO of Twitter is Jack Dorsey.'

Let's implement a simple search engine that will be able to answer questions about an evolving topic. In the following example, we will parse the Google result page. It provides typically 10 results, sometimes prefaced by a “featured snippet”  followed by other questions that “people also ask”, and at the end of the list of results, a few “related searches”. We will then check the number of tokens on the page to make sure that it isn’t too long to “stuff” it with the initial question into a prompt for ChatGPT.

In [13]:
from bs4 import BeautifulSoup
import requests

prompt = 'who is the CEO of twitter?'

def search(prompt):
    url = f'https://www.google.com/search?q={prompt}'
    html = requests.get(url).text
    with open('search.html','w') as f:
        f.write(html)
    # Get the text of the webpage
    soup = BeautifulSoup(html, "html.parser")
    text = soup.get_text()
    return text
text = search(prompt)
len(text)

8162

In [8]:
import tiktoken
def num_tokens(string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding_name = 'cl100k_base'
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens
num_tokens(text)

2116

The google result page is typically dense enough that we can simply stuff it into a model and get a good answer. Sometimes, you might want to retrieve the top 3 or 5 pages from the search to get a more comprehensive answer. 

In [12]:
question = f"""Given the following context of Google search, answer the question:
{prompt}
---
Here is the context retrieve from Google search:
{text}
"""
ask(question)

'The CEO of Twitter is Linda Yaccarino.'

## Vector search

Example: First paragraph of chapter 1 of [A Tale of Two Cities by Charles Dickens](https://www.gutenberg.org/ebooks/98)


In [6]:
paragraph = """
It was the best of times, it was the worst of times, it was the age of
wisdom, it was the age of foolishness, it was the epoch of belief, it
was the epoch of incredulity, it was the season of Light, it was the
season of Darkness, it was the spring of hope, it was the winter of
despair, we had everything before us, we had nothing before us, we were
all going direct to Heaven, we were all going direct the other way--in
short, the period was so far like the present period, that some of its
noisiest authorities insisted on its being received, for good or for
evil, in the superlative degree of comparison only.
"""
sentences = paragraph.replace("\n"," ").split(", ")
sentences

[' It was the best of times',
 'it was the worst of times',
 'it was the age of wisdom',
 'it was the age of foolishness',
 'it was the epoch of belief',
 'it was the epoch of incredulity',
 'it was the season of Light',
 'it was the season of Darkness',
 'it was the spring of hope',
 'it was the winter of despair',
 'we had everything before us',
 'we had nothing before us',
 'we were all going direct to Heaven',
 'we were all going direct the other way--in short',
 'the period was so far like the present period',
 'that some of its noisiest authorities insisted on its being received',
 'for good or for evil',
 'in the superlative degree of comparison only. ']

In [3]:
sentences = [
    "it caught him off guard that space smelled of seared steak",
    "she could not decide between painting her teeth or brushing her nails",
    "he thought there'd be sufficient time is he hid his watch",
    "the bees decided to have a mutiny against their queen",
    "the sign said there was road work ahead so she decided to speed up",
    "on a scale of one to ten, what's your favorite flavor of color?",
    "flying stinging insects rebelled in opposition to the matriarch"
]

In [7]:
from openai import OpenAI
client = OpenAI()

sentence = sentences[0]

response = client.embeddings.create(
    input=sentence,
    model="text-embedding-3-small"
)

embedding = response.data[0].embedding
print(len(embedding))
embedding[:3]

1536


[-0.007199400570243597, -0.01370294764637947, 0.02629520744085312]

In [8]:
import numpy as np
response = client.embeddings.create(
    input=sentences,
    model="text-embedding-3-small"
)
v = [d.embedding for d in response.data]
v = np.array(v)
v[:3,:3]

array([[-7.19940057e-03, -1.37029476e-02,  2.62952074e-02],
       [-2.73702480e-02, -1.38540762e-02,  4.92458791e-02],
       [ 4.99483123e-02,  4.94058040e-05,  6.35890104e-03]])

In [9]:
v.shape

(18, 1536)

Cosine similarity

In [10]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

cosine_similarity(v[0], v[1])

0.674606291470473

In [11]:
sentences[0], sentences[1], cosine_similarity(v[0], v[1])

(' It was the best of times', 'it was the worst of times', 0.674606291470473)

In [12]:
sentences[2], sentences[3], cosine_similarity(v[2], v[3])

('it was the age of wisdom',
 'it was the age of foolishness',
 0.8072276532681985)

scikit-learn provides the cosine_similarity function in the sklearn.metrics.pairwise module

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

# Assuming vec1 and vec2 are 1-D numpy arrays
cosine_similarity([v[0]],[v[1]])[0][0]

0.6746062914704734

SciPy provides the spatial.distance.cosine function to compute the cosine distance, which can be converted to similarity by subtracting from 1.

In [27]:
from scipy.spatial.distance import cosine

similarity = 1 - cosine(v[0], v[1])
similarity

0.674606291470473

Alternative embedding models: Sentence Transformers

In [ ]:
%pip install -U sentence-transformers

In [28]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2') # or 'all-mpnet-base-v2'

# Sentences are encoded by calling model.encode()
embedding = model.encode(sentence)
embedding

modules.json: 100%|██████████| 349/349 [00:00<?, ?B/s] 
c:\Users\ydebray\Downloads\gpt-programming-book\env\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ydebray\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
config_sentence_transformers.json: 100%|██████████| 116/116 [00:

array([-4.52113152e-02,  7.17497021e-02,  1.72493700e-02, -3.16724032e-02,
        4.76625152e-02, -8.03213287e-03,  1.21740187e-02,  5.78135513e-02,
       -2.35217176e-02,  5.41551213e-04, -5.85709698e-03,  1.64452136e-01,
        8.72123465e-02,  5.75544350e-02, -9.70988721e-03,  2.56951135e-02,
        3.18162329e-02,  4.86325705e-03, -2.24988852e-02, -3.80984396e-02,
       -5.67632802e-02, -2.70305481e-02,  1.22782504e-02,  4.93002757e-02,
        9.22550075e-03,  3.00662238e-02,  7.22770486e-03,  6.36119768e-02,
       -2.89241765e-02,  3.43357697e-02, -3.90363559e-02, -2.01459415e-02,
        9.26952867e-04,  1.15649151e-02, -5.66930957e-02,  3.26514547e-03,
        1.08313095e-02, -6.76954165e-02,  2.46112198e-02, -8.44309398e-04,
        3.83343524e-03, -3.42086218e-02, -2.42658071e-02, -8.44405144e-02,
        1.69295315e-02,  3.31159658e-03,  8.05927143e-02, -8.86978805e-02,
        5.58701269e-02,  2.43993625e-02,  4.77015637e-02, -1.13933217e-02,
        4.62255739e-02, -

In [31]:
embedding.shape

(384,)

In [37]:
from sentence_transformers import util

vec1 = model.encode(sentences[0])
vec2 = model.encode(sentences[1])

similarity = util.pytorch_cos_sim(vec1, vec2).tolist()
similarity[0][0],sentences[0],sentences[1]

(0.7910879850387573, ' It was the best of times', 'it was the worst of times')

## LlamaIndex: building an index


- Step 1: Load data (PDF reader)

In [27]:
import requests, io, pypdf
# get the impromptu book
url = 'https://www.impromptubook.com/wp-content/uploads/2023/03/impromptu-rh.pdf'

def pdf_to_pages(file):
	"extract text (pages) from pdf file"
	pages = []
	pdf = pypdf.PdfReader(file)
	for p in range(len(pdf.pages)):
		page = pdf.pages[p]
		text = page.extract_text()
		pages += [text]
	return pages

r = requests.get(url)
f = io.BytesIO(r.content)
pages = pdf_to_pages(f)
print(pages[1])

Impromptu
Amplifying Our Humanity 
Through AI
By Reid Hoffman  
with GPT-4


Let's save the content of the PDF into txt files.

In [28]:
if not os.path.exists("impromptu"):
    os.mkdir("impromptu")
for i, page in enumerate(pages):
    with open(f"impromptu/{i}.txt","w", encoding='utf-8') as f:
        f.write(page)

In [29]:
sep = '\n'
book = sep.join(pages)
print(book[0:35])


Impromptu
Amplifying Our Humanity 


In [30]:
import tiktoken
def num_tokens(string):
    """Returns the number of tokens in a text string."""
    encoding_name = 'cl100k_base'
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens(book)

83310

- Step 2: Build an index

In [32]:
from llama_index import SimpleDirectoryReader, VectorStoreIndex
documents = SimpleDirectoryReader("impromptu").load_data()
index = VectorStoreIndex.from_documents(documents)
documents[1]

Document(id_='864d60cc-71a5-48c8-9692-a837cd438ce8', embedding=None, metadata={'file_path': 'impromptu\\1.txt', 'file_name': '1.txt', 'file_type': 'text/plain', 'file_size': 78, 'creation_date': '2023-12-04', 'last_modified_date': '2024-04-22', 'last_accessed_date': '2024-04-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, hash='54b42932d386d64b974600eabc107b410fd93cf33967e87617626d44bec310f3', text='Impromptu\nAmplifying Our Humanity \nThrough AI\nBy Reid Hoffman  \nwith GPT-4', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

In [33]:
documents[1].dict()

{'id_': '864d60cc-71a5-48c8-9692-a837cd438ce8',
 'embedding': None,
 'metadata': {'file_path': 'impromptu\\1.txt',
  'file_name': '1.txt',
  'file_type': 'text/plain',
  'file_size': 78,
  'creation_date': '2023-12-04',
  'last_modified_date': '2024-04-22',
  'last_accessed_date': '2024-04-22'},
 'excluded_embed_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'excluded_llm_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'relationships': {},
 'hash': '54b42932d386d64b974600eabc107b410fd93cf33967e87617626d44bec310f3',
 'text': 'Impromptu\nAmplifying Our Humanity \nThrough AI\nBy Reid Hoffman  \nwith GPT-4',
 'start_char_idx': None,
 'end_char_idx': None,
 'text_template': '{metadata_str}\n\n{content}',
 'metadata_template': '{key}: {value}',
 'metadata_seperator': '\n',
 'class_name': 'Document'}

In [34]:
# save to disk
index.storage_context.persist()

- Step 3: Query the index

In [35]:
query_engine = index.as_query_engine()
response = query_engine.query('what is the potential of AI in education?')
print(response)

The potential of AI in education is significant. It has the capability to transform the way we learn and deliver instruction by providing personalized, individualized learning experiences tailored to each student's needs and interests. AI can identify the topics and skills students need to focus on, offer guidance and support, and enable more direct interaction between students and teachers for engaging and meaningful instruction. Additionally, AI-driven tools can automate mundane teaching tasks like grading and content creation, allowing teachers more time to focus on inspiring their students.


In [36]:
response.get_formatted_sources()

'> Source (Doc id: 621f0645-12b6-4422-bfc9-208ba5f1409c): 47Education\nthe technology will also create an educational system \nthat is less equitable and acc...\n\n> Source (Doc id: 90b23e79-3feb-4fc3-b511-4c81f7e34dc5): 46Impromptu: Amplifying Our Humanity Through AI\nReid: GPT-4, there are so many more subjects I wa...'

In [37]:
sources = [s.node.get_text() for s in response.source_nodes]
# print(len(sources))
print(sources[0][0:11])

47Education


- Optional: Look under the hood of the index

In [39]:
import os
os.listdir('storage')

['default__vector_store.json',
 'docstore.json',
 'graph_store.json',
 'index_store.json']

Documents are assigned unique identifiers like `864d60cc-71a5-48c8-9692-a837cd438ce8`

In [ ]:
import json
with open('storage/docstore.json','r') as f:
    docstore = json.load(f)
docstore.keys()

dict_keys(['docstore/metadata', 'docstore/data', 'docstore/ref_doc_info'])

In [ ]:
docstore['docstore/ref_doc_info']['864d60cc-71a5-48c8-9692-a837cd438ce8']

{'node_ids': ['6b45f3cd-7391-4631-a6b0-dbdd65de5f26'],
 'metadata': {'file_path': 'impromptu\\1.txt',
  'file_name': '1.txt',
  'file_type': 'text/plain',
  'file_size': 78,
  'creation_date': '2023-12-04',
  'last_modified_date': '2024-04-22',
  'last_accessed_date': '2024-04-22'}}

In [63]:
docstore['docstore/metadata']['6b45f3cd-7391-4631-a6b0-dbdd65de5f26']

{'doc_hash': '8a044acf635232523b775ad356f1d9af9edfcf497b8daaec2af8ca3685c6f878',
 'ref_doc_id': '864d60cc-71a5-48c8-9692-a837cd438ce8'}

In [64]:
docstore['docstore/data']['6b45f3cd-7391-4631-a6b0-dbdd65de5f26']

{'__data__': {'id_': '6b45f3cd-7391-4631-a6b0-dbdd65de5f26',
  'embedding': None,
  'metadata': {'file_path': 'impromptu\\1.txt',
   'file_name': '1.txt',
   'file_type': 'text/plain',
   'file_size': 78,
   'creation_date': '2023-12-04',
   'last_modified_date': '2024-04-22',
   'last_accessed_date': '2024-04-22'},
  'excluded_embed_metadata_keys': ['file_name',
   'file_type',
   'file_size',
   'creation_date',
   'last_modified_date',
   'last_accessed_date'],
  'excluded_llm_metadata_keys': ['file_name',
   'file_type',
   'file_size',
   'creation_date',
   'last_modified_date',
   'last_accessed_date'],
  'relationships': {'1': {'node_id': '864d60cc-71a5-48c8-9692-a837cd438ce8',
    'node_type': '4',
    'metadata': {'file_path': 'impromptu\\1.txt',
     'file_name': '1.txt',
     'file_type': 'text/plain',
     'file_size': 78,
     'creation_date': '2023-12-04',
     'last_modified_date': '2024-04-22',
     'last_accessed_date': '2024-04-22'},
    'hash': '54b42932d386d64b9746

In [65]:
docstore['docstore/data']['6b45f3cd-7391-4631-a6b0-dbdd65de5f26']

{'__data__': {'id_': '6b45f3cd-7391-4631-a6b0-dbdd65de5f26',
  'embedding': None,
  'metadata': {'file_path': 'impromptu\\1.txt',
   'file_name': '1.txt',
   'file_type': 'text/plain',
   'file_size': 78,
   'creation_date': '2023-12-04',
   'last_modified_date': '2024-04-22',
   'last_accessed_date': '2024-04-22'},
  'excluded_embed_metadata_keys': ['file_name',
   'file_type',
   'file_size',
   'creation_date',
   'last_modified_date',
   'last_accessed_date'],
  'excluded_llm_metadata_keys': ['file_name',
   'file_type',
   'file_size',
   'creation_date',
   'last_modified_date',
   'last_accessed_date'],
  'relationships': {'1': {'node_id': '864d60cc-71a5-48c8-9692-a837cd438ce8',
    'node_type': '4',
    'metadata': {'file_path': 'impromptu\\1.txt',
     'file_name': '1.txt',
     'file_type': 'text/plain',
     'file_size': 78,
     'creation_date': '2023-12-04',
     'last_modified_date': '2024-04-22',
     'last_accessed_date': '2024-04-22'},
    'hash': '54b42932d386d64b9746

In [ ]:
with open('storage/default__vector_store.json','r') as f:
    data = json.load(f)
data.keys()

dict_keys(['embedding_dict', 'text_id_to_ref_doc_id', 'metadata_dict'])

In [69]:
len(data['embedding_dict']['6b45f3cd-7391-4631-a6b0-dbdd65de5f26'])

1536

## Vector databases

Let's try ChromaDB, and install all necessary dependencies
```
pip install -U langchain langchain-openai unstructured chromadb
```

In [1]:
import chromadb
# client = chromadb.HttpClient()
client = chromadb.PersistentClient()
collection = client.create_collection("sample_collection")

# Add docs to the collection. Can also update and delete. Row-based API coming soon!
collection.add(
    documents=["This is document1", "This is document2"], # we embed for you, or bring your own
    metadatas=[{"source": "notion"}, {"source": "google-docs"}], # filter on arbitrary metadata!
    ids=["doc1", "doc2"], # must be unique for each doc 
)

results = collection.query(
    query_texts=["This is a query document"],
    n_results=2,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    # where_document={"$contains":"search_string"}  # optional filter
)  
results

C:\Users\ydebray\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:08<00:00, 10.3MiB/s]


{'ids': [['doc1', 'doc2']],
 'distances': [[0.9026352763807001, 1.0358158255050436]],
 'metadatas': [[{'source': 'notion'}, {'source': 'google-docs'}]],
 'embeddings': None,
 'documents': [['This is document1', 'This is document2']],
 'uris': None,
 'data': None}

Integration in LangChain: Example with chapter 1 of Impromptu on Education (not using the full book to avoid unnecessary cost to create embeddings)

In [19]:
# Retrieve the pdf and extract chap 32-54
import requests, io, pypdf
url = 'https://www.impromptubook.com/wp-content/uploads/2023/03/impromptu-rh.pdf'
r = requests.get(url)
f = io.BytesIO(r.content)
pdf = pypdf.PdfReader(f)
writer = pypdf.PdfWriter()
for p in range(31,54):
    writer.add_page(pdf.pages[p])
with open("impromptu_32-54.pdf","wb") as f:
    writer.write(f)

In [ ]:
# Extract text from the pdf
pages = []
for p in range(32,55):
    page = pdf.pages[p]
    text = page.extract_text()
    pages += [text]

print(pages[0][0:48])

In [20]:
from langchain.chains import RetrievalQA
# from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain_community.document_loaders import PyPDFLoader
# from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
chat = ChatOpenAI(model_name='gpt-3.5-turbo')

In [22]:
# Load the document and split it into pages
loader = PyPDFLoader("impromptu_32-54.pdf")
# loader = TextLoader('impromptu/53.txt')
# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# texts = text_splitter.split_documents(documents)
# loader = DirectoryLoader('impromptu')
pages = loader.load_and_split()

In [23]:
# select which embeddings we want to use
embeddings = OpenAIEmbeddings()
# create the vectorestore to use as the index
db = Chroma.from_documents(pages, embeddings)

In [24]:
# expose this index in a retriever interface
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":3})
# create a chain to answer questions 
qa = RetrievalQA.from_chain_type(
    llm=chat, chain_type="stuff", retriever=retriever, return_source_documents=True)
query = 'what are the opportunities of using AI?'
result = qa.invoke({"query": query})

In [25]:
result

{'query': 'what are the opportunities of using AI?',
 'result': 'The opportunities of using AI in education include automating and streamlining mundane tasks like grading and content creation, providing personalized and individualized learning experiences, giving teachers more time to focus on engaging students, and potentially transforming the way we learn and deliver instruction. AI can also help identify topics and skills students need to focus on and provide guidance and support accordingly.',
 'source_documents': [Document(page_content='47Education\nthe technology will also create an educational system \nthat is less equitable and accessible.\nMixed: Large language models will have an undeni-\nable impact on education in the next fifty years, but its \npotential for transforming the way we learn and deliver \ninstruction will depend on a range of factors. AI-driven \ntools will be used to automate and streamline some of \nthe more mundane aspects of teaching, such as grading \nand

Let’s try Facebook AI Similarity Search (faiss), which is known to be insanely performant:
```
pip install faiss-cpu
```	

In [28]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())
docs = faiss_index.similarity_search("what are the opportunities of using AI?", k=3)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:48])

22: 47Education
the technology will also create an e
21: 46Impromptu: Amplifying Our Humanity Through AI

3: 28Impromptu: Amplifying Our Humanity Through AI



More about FAISS

In [ ]:
import numpy as np
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

In [ ]:
import faiss                   # make faiss available
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

True
100000


In [ ]:
k = 4                          # we want to see 4 nearest neighbors
D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)
D, I = index.search(xq, k)     # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])                  # neighbors of the 5 last queries

[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]
[[0.        7.1751733 7.2076297 7.2511625]
 [0.        6.3235645 6.684581  6.799946 ]
 [0.        5.7964087 6.391736  7.2815123]
 [0.        7.2779055 7.5279875 7.662846 ]
 [0.        6.7638035 7.2951202 7.368815 ]]
[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]


## Bonus

In [2]:
from llama_index import LLMPredictor, ServiceContext
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
chat = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
llm_predictor = LLMPredictor(llm=chat)
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)
res = chat([HumanMessage(content='What does impromptu mean?')])
res.dict()['content']

'Impromptu means done without advance preparation or planning; spur of the moment.'

In [ ]:

# from llama_index import download_loader
# PDFReader = download_loader("PDFReader")
# loader = PDFReader()
# documents = loader.load_data(pdf)
# documents